In [1]:
# Import packages

import numpy as np
import pandas as pd
import h5py  
import itertools
import os
import datetime
from dateutil.relativedelta import relativedelta


In [ ]:
# Read Excel spreadsheets of sewage overflow events

SRdf= pd.read_csv("raincause_ovflow.csv")
SNRdf = pd.read_csv("othercauses_ovflow.csv") 
Sdf = SRdf.append(SNRdf)
print(Sdf.head())
print(Sdf.info())


In [ ]:
# Drop columns

Sdf = Sdf.drop('Unnamed: 0', axis=1)  # remove "Unnamed" column
Sdf.rename(columns={'LATTITUDE DECIMAL DEGREES':'LAT'}, inplace=True)
Sdf.rename(columns={'LONGITUDE DECIMAL DEGREES':'LON'}, inplace=True)
Sdf.rename(columns={'SPILL ID':'SPILL_ID'}, inplace=True)
Sdf.rename(columns={'START DT':'START_DATE'}, inplace=True)
Sdf.rename(columns={'SPILL CAUSE':'SPILL_CAUSE'}, inplace=True)
Sdf.rename(columns={'MATERIAL_SEWER_PIPE':'MATERIAL'}, inplace=True)
Sdf.rename(columns={'AGE_SEWER_PIPE':'AGE'}, inplace=True)
Sdf.rename(columns={'DIAMETER_OF_SEWER':'DIAMETER'}, inplace=True)

print(Sdf.head())


In [ ]:
# Filter for after March 31, 2015 (where we have SMAP data)

Sdf = Sdf[(Sdf['start_date'] > '2015-03-31')]
Sdf = Sdf[(Sdf['lon'] < -100)]  # drop entries with out-of-bounds longitude
Sdf = Sdf.sort_values('start_date')
Sdf = Sdf.reset_index(drop=True) 
print(Sdf.info())
print(Sdf.head())
print(Sdf.tail())


In [ ]:
# Convert Pacific local times to UTC, accounting for Daylight Savings Time

Sdf['start_date'] = pd.to_datetime(Sdf['start_date'])
start_dates = Sdf['start_date']

for j in range(len(start_dates)) :
    if start_dates[j] < pd.to_datetime('11/1/2015 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=7)
    elif start_dates[j] < pd.to_datetime('3/13/2016 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=8)
    elif start_dates[j] < pd.to_datetime('11/6/2016 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=7)
    elif start_dates[j] < pd.to_datetime('3/12/2017 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=8)        
    elif start_dates[j] < pd.to_datetime('11/5/2017 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=7)                                                     
    elif start_dates[j] < pd.to_datetime('3/11/2018 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=8)
    elif start_dates[j] < pd.to_datetime('11/4/2018 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=7)        
    elif start_dates[j] < pd.to_datetime('3/10/2019 02:00') : 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=8)        
    else: 
        Sdf.loc[j,'UTC_time'] = start_dates[j] + relativedelta(hours=7)        
 
        

In [ ]:
# Re-order columns

Sdf = Sdf[['SPILL_ID', 'START_DATE', 'UTC_time', 'LAT', 'LON', 'SPILL_CAUSE', 'SPILL_TYPE', 'SPILL_VOL', 'MATERIAL', 'AGE', 'DIAMETER']]

Sdf



In [ ]:
# Calculate time of first SMAP data file, which are released every three hours and are dated in the middle of that period
# (e.g., data for 9pm to midnight is in file ending "T223000" for 10:30pm)

def round_down(hour):
    while hour % 3 != 0 : 
        hour = hour - 1
    else : 
        return hour

Sdf['datestring'] = Sdf['UTC_time'].apply(lambda x:x.strftime('%Y%m%d'))
Sdf['hour'] = Sdf['UTC_time'].dt.hour

x = 0 
while x < Sdf.LAT.count():  
    # round hour down, add minutes + 90, then seconds to string
    Sdf.loc[x,'timestring'] = str(round_down(Sdf.loc[x,'hour']) * 100 + 130) + '00' 
    if len(Sdf.loc[x,'timestring']) < 6 :
        Sdf.loc[x,'timestring'] = '0' + Sdf.loc[x,'timestring']  # pad zero for AM files
    # create filename and URL
    Sdf.loc[x,'filename'] = 'SMAP_L4_SM_gph_' + Sdf.loc[x,'datestring'] + 'T' + Sdf.loc[x,'timestring'] + '_Vv4030_001.h5'
    Sdf.loc[x,'datetime'] = Sdf.loc[x,'datestring'] + ' ' + Sdf.loc[x,'timestring']
    
    x = x + 1 

Sdf['datetime'] = pd.to_datetime(Sdf['datetime'])


In [ ]:
# Drop unneeded columns

try:
    Sdf = Sdf.drop(['datestring'], axis=1)
    Sdf = Sdf.drop(['timestring'], axis=1)
    Sdf = Sdf.drop(['hour'], axis=1)
except:
    print('Columns already deleted')
    

In [ ]:
# Need that file/date and previous 31 files (96 hours, or 4 days)
# So multiply df by 32

Sdf = pd.concat([Sdf]*32, ignore_index=True)
Sdf = Sdf.sort_values('SPILL_ID')
Sdf = Sdf.reset_index(drop=True)


In [ ]:
# Then write proper datetimes for each 3-hr slot

x = 0

while x < len(Sdf['SPILL_ID']) :
    Sdf.loc[x,'datetime-4d'] = Sdf.loc[x,'datetime'] - relativedelta(days=4)
    dr = pd.date_range(Sdf.loc[x,'datetime-4d'], periods=32, freq='3H')
    y = 0
    while y < 31 :
        Sdf.loc[x+y,'datetime'] = dr[y+1]
        y = y + 1
        
    x = x + 32



In [ ]:
# And then re-write filenames

Sdf['datestring'] = Sdf['datetime'].apply(lambda x:x.strftime('%Y%m%d'))
Sdf['timestring'] = Sdf['datetime'].apply(lambda x:x.strftime('%H%M%S'))

x = 0 
while x < Sdf.LAT.count():  

    Sdf.loc[x,'filename'] = 'SMAP_L4_SM_gph_' + Sdf.loc[x,'datestring'] + 'T' + Sdf.loc[x,'timestring'] + '_Vv4030_001.h5'
    print( str(x) + "   Spill ID: " + str(Sdf.loc[x,'SPILL_ID']) + "   File: " + Sdf.loc[x,'filename'] )
    
    x = x + 1 


In [ ]:
# Drop unneeded columns

try:
    Sdf = Sdf.drop(['datestring'], axis=1)
    Sdf = Sdf.drop(['timestring'], axis=1)
    Sdf = Sdf.drop(['datetime-4d'], axis=1)
except:
    print('Columns already deleted')
    

In [ ]:
# Drop spills before Apr 4, 2015 because we are going back 4 days

Sdf = Sdf[Sdf['UTC_time'] > '4/4/2015']


In [3]:
# Make list of filenames

filenames = Sdf['filename'].tolist()
print( str(len(filenames)) + " filenames/entries")
filenames = list(dict.fromkeys(filenames)) # remove duplicates
print( str(len(filenames)) + " unique files")


438272 filenames/entries
11591 unique files


In [7]:
# Make batch

filebatch = filenames[3000:5134]


In [9]:
# Scrape files for SMAP data

for filename in filebatch :

    filepath = "D:/SMAP_Data/" + str(filename)
    print(filepath)
    
    try:
           
        with h5py.File(filepath, mode='r') as f:
      
            sm_surface = f['Geophysical_Data']['sm_surface'][:]
            sm_surface_wetness = f['Geophysical_Data']['sm_surface_wetness'][:]  # dimensionless
            sm_rootzone = f['Geophysical_Data']['sm_rootzone'][:]
            sm_rootzone_wetness = f['Geophysical_Data']['sm_rootzone_wetness'][:]
            sm_profile = f['Geophysical_Data']['sm_profile'][:]
            sm_profile_wetness = f['Geophysical_Data']['sm_profile_wetness'][:]

            vegetation_greenness_fraction = f['Geophysical_Data']['vegetation_greenness_fraction'][:]
            leaf_area_index = f['Geophysical_Data']['leaf_area_index'][:]
        
            land_fraction_saturated = f['Geophysical_Data']['land_fraction_saturated'][:]
            land_fraction_unsaturated = f['Geophysical_Data']['land_fraction_unsaturated'][:]
            land_fraction_snow_covered = f['Geophysical_Data']['land_fraction_snow_covered'][:]
            land_fraction_wilting = f['Geophysical_Data']['land_fraction_wilting'][:]
        
            surface_temp = f['Geophysical_Data']['surface_temp'][:]
            surface_pressure = f['Geophysical_Data']['surface_pressure'][:]
        
            heat_flux_ground = f['Geophysical_Data']['heat_flux_ground'][:]
            heat_flux_latent = f['Geophysical_Data']['heat_flux_latent'][:]
            heat_flux_sensible = f['Geophysical_Data']['heat_flux_sensible'][:]
            land_evapotranspiration_flux = f['Geophysical_Data']['land_evapotranspiration_flux'][:]
            overland_runoff_flux = f['Geophysical_Data']['overland_runoff_flux'][:]
            soil_water_infiltration_flux = f['Geophysical_Data']['soil_water_infiltration_flux'][:]
   
            lat = f['cell_lat'][:]
            lat_1d = lat[:,0]   # first column of lat (all other rows the same)
            lon = f['cell_lon'][:]
            lon_1d = lon[0]  # first row of lon (all other rows the same)
    
            # need to get > 1 datasets from each file, so choose the rows 
            rows = Sdf.loc[Sdf['filename']==filename] 
#            print(rows)
    
            for index, row in rows.iterrows():  # grab (almost) everything ;)
            
                lat_index = (np.abs(lat_1d - rows.loc[index,'LAT'])).argmin()  # find index closest to spill latitude
                lon_index = (np.abs(lon_1d - rows.loc[index,'LON'])).argmin()  # find index closest to spill longitude
#                print("Lat index: " + str(lat_index) + "      Lon index: " + str(lon_index) )
          
                Sdf.loc[index,'sm_surface'] = sm_surface[lat_index, lon_index]
                Sdf.loc[index,'sm_surface_wetness'] = sm_surface_wetness[lat_index, lon_index]
                Sdf.loc[index,'sm_rootzone'] = sm_rootzone[lat_index, lon_index]
                Sdf.loc[index,'sm_rootzone_wetness'] = sm_rootzone_wetness[lat_index, lon_index]
                Sdf.loc[index,'sm_profile'] = sm_profile[lat_index, lon_index]
                Sdf.loc[index,'sm_profile_wetness'] = sm_profile_wetness[lat_index, lon_index]

                Sdf.loc[index,'vegetation_greenness_fraction'] = vegetation_greenness_fraction[lat_index, lon_index]
                Sdf.loc[index,'leaf_area_index'] = leaf_area_index[lat_index, lon_index]
            
                Sdf.loc[index,'land_fraction_saturated'] = land_fraction_saturated[lat_index, lon_index]
                Sdf.loc[index,'land_fraction_unsaturated'] = land_fraction_unsaturated[lat_index, lon_index]
                Sdf.loc[index,'land_fraction_snow_covered'] = land_fraction_snow_covered[lat_index, lon_index]
                Sdf.loc[index,'land_fraction_wilting'] = land_fraction_wilting[lat_index, lon_index]

                Sdf.loc[index,'surface_temp'] = surface_temp[lat_index, lon_index]
                Sdf.loc[index,'surface_pressure'] = surface_pressure[lat_index, lon_index]    
                 
                Sdf.loc[index,'heat_flux_ground'] = heat_flux_ground[lat_index, lon_index]
                Sdf.loc[index,'heat_flux_latent'] = heat_flux_latent[lat_index, lon_index]
                Sdf.loc[index,'heat_flux_sensible'] = heat_flux_sensible[lat_index, lon_index]
                Sdf.loc[index,'land_evapotranspiration_flux'] = land_evapotranspiration_flux[lat_index, lon_index]
                Sdf.loc[index,'overland_runoff_flux'] = overland_runoff_flux[lat_index, lon_index]
                Sdf.loc[index,'soil_water_infiltration_flux'] = soil_water_infiltration_flux[lat_index, lon_index]
 
        # close file
        f.close()
        
    except:
        print("Couldn't read file...............")
        


D:/SMAP_Data/SMAP_L4_SM_gph_20160409T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160409T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160410T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160411T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160411T043000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20160426T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160427T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160428T223000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20160514T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160514T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160514T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160515T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160516T163000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20160601T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160601T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160601T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160601T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160601T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160602T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160603T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160603T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160603T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160603T103000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20160619T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160619T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160620T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160621T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20160621T043000_Vv4030_001.h5
D:/SMAP_Data/S

...

D:/SMAP_Data/SMAP_L4_SM_gph_20161021T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161021T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161021T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161021T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161021T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161022T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161023T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161023T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161023T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161023T103000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20161108T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161108T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161109T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161110T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161110T043000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20161125T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161126T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161127T223000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20161213T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161213T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161213T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T193000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161214T223000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T013000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T043000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T073000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161215T163000_Vv4030_001.h5
D:/SMAP_Data/S

D:/SMAP_Data/SMAP_L4_SM_gph_20161231T103000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161231T133000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161231T163000_Vv4030_001.h5
D:/SMAP_Data/SMAP_L4_SM_gph_20161231T193000_Vv4030_001.h5


In [11]:
# Write batch

Sdf.to_csv('Sdf_0-5133[Dec2016].csv')


In [ ]:
print(Sdf.info())
Sdf